In [ ]:
# default_exp sh

# mrtrix-format spherical harmonics image handling

also see: https://github.com/MRtrix3/mrtrix3/pull/1617 extends SH storage to allow storing SH along the 5th dimension of an image, one per shell. shconv now allows multiple ODF/response pairs (much like dwi2fod and mtnormalise do), and allows multi-shell responses (in which case it'll produce a 5D output). sh2amp can now accepts directions files in spherical or Cartesian coordinates, full-blown DW encoding files, or an image with a DW encoding in its header (required to handle multi-shell inputs).

This allows forward-modelling of all our spherical deconvolution operations, e.g. for MSMT-CSD:

shconv wm.mif wm.txt gm.mif gm.txt csf.mif csf.txt - | sh2amp - dwi_orig.mif dwi_predicted.mif Also in there are a few changes to docs handling, and minor updates to the tests.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

'''
process spherical harmonics images in MRtrix format.


'''


def sh2power(R, lmax):
    import numpy as np
    def index (l, m):
        return int(l * (l+1) / 2 + m)
    
    power = np.zeros(list(R.shape[:3])+[lmax//2+1], dtype=np.float64)
    for il, l in enumerate(range(0, lmax+2, 2)):
        for m in range(-l, l+1):
            idx = index (l, m)
            power[...,il] += R[...,idx]**2
        power[...,il] /= (np.pi * 4)
    return power


def sh2spectrum(R, lmax=6):
    """ L2 norm (mean and σ): expectation and square root of sum of squares in each frequency band (order)"""
    import numpy as np
    # https://en.wikipedia.org/wiki/Spherical_harmonics#Power_spectrum_in_signal_processing
    # square root of energy https://www.dsprelated.com/freebooks/mdft/Signal_Metrics.html
    return np.sqrt(sh2power(R, lmax) * np.pi * 4)


def sh2RISH(R, lmax):
    """sum of squares in each frequency band (order)"""
    import numpy as np
    # Kazhdan, M., Funkhouser, T., Rusinkiewicz, S.: Rotation invariant spherical harmonic representation of 3D shape descriptors. Symposium on Geometry Processing (2003)
    return sh2power(R, lmax) * np.pi * 4


def amp2shellagg(dwi, fun='mean', grad=None, btol=50.):
    import numpy as np
    if fun == 'mean':
        fun = np.mean
    else:
        raise NotImplementedError(str(fun))
        
    if isinstance(dwi, str):
        dwi = load_mrtrix(dwi)
        data = dwi.data
        if grad is None:
            grad = dwi.grad
    elif isinstance(dwi, np.ndarray):
        data = dwi
    
    if grad is None:
        raise ValueError('gradient table required')

    bs = grad[:,3]
    if btol > 0:
        bs = np.round(bs / float(btol)) * btol
    ubs = sorted(np.unique(bs).tolist())
    

    agg = np.zeros(list(dwi.shape[:3])+[len(ubs)], dtype=np.float32)
    for ib, b in enumerate (ubs):
        agg[...,ib] = fun(data[...,bs==b], axis=3)

    return agg

import tempfile
def amp2mssh(dwi, grad=None, btol=50., lmax_callback=None):
    from mrtrix3nb.utils.mif import Image, load_mrtrix
    from mrtrix3nb.core import Comp
    import numpy as np
    """returns list with one sh series per b-value"""
    shs = []
    with tempfile.TemporaryDirectory() as tmpdirname:
        with Comp(loglevel='ERROR', progress=False) as run:
            if isinstance(dwi, str):
                im = load_mrtrix(dwi, read_data=True)
                if grad is not None:
                    im.grad = grad

                if im.grad is None:
                    raise ValueError('gradient table required for', dwi)
                im.save(tmpdirname+'/dwi.mif')
                dwi = tmpdirname+'/dwi.mif'

            elif isinstance(dwi, np.ndarray):
                if grad is None:
                    raise ValueError('gradient table required')
                im = Image(data=dwi, grad=grad)
                im.save(tmpdirname+'/dwi.mif')
                dwi = tmpdirname+'/dwi.mif'

            bs = im.grad[:,3]
            if btol > 0:
                bs = np.round(bs / float(btol)) * btol
            ubs = sorted(np.unique(bs).tolist())
            
            shs = []
            for ib, b in enumerate(ubs):
                lmax = ''
                if lmax_callback is not None:
                    lmax = ' -lmax ' + str(lmax_callback(im.grad[bs==b]))
                if b >= 100:
                    run += f'amp2sh {dwi} -shell {b} {tmpdirname}/sh{ib}.mif' + lmax
                else:
                    run += f'dwiextract {dwi} -shell {b} - | mrmath - mean -axis 3 - | mrcalc - 1.77245385091 -mult 2 -mult - | mrconvert - -axes 0:2,-1 {tmpdirname}/sh{ib}.mif'
                shs.append(f'{tmpdirname}/sh{ib}.mif')
        
        for ish in range(len(shs)):
            shs[ish] = load_mrtrix(shs[ish]).data

    return shs

In [ ]:
#hide
dwi = '/Users/mp/data/tmp/postmc_dstriped-dwi300.mif'
shs = amp2mssh(dwi)
shs[-1].shape

(100, 100, 64, 45)

In [ ]:
# !make